In [1]:
import pandas as pd
import glob
from datetime import timedelta, date

In [2]:
archivos = glob.glob('.\INPUT\ZOOM_C4\*.csv')
dfsBase = []
for archivo in archivos:
    dfsBase.append(pd.read_csv(archivo))
DATOS = pd.concat(dfsBase, ignore_index=True)

In [3]:
INFO_FORMADORES = pd.read_excel('.\INPUT\INFO_SALAS_FORMADORES.xlsx', engine='openpyxl')

In [4]:
DIAS_LABORALES = pd.bdate_range('10/02/2022', '11/12/2022')

In [5]:
DATOS['Hora de inicio'] = pd.to_datetime(DATOS['Hora de inicio'], format="%m/%d/%Y %H:%M:%S")
DATOS['Hora de finalización'] = pd.to_datetime(DATOS['Hora de finalización'], format="%m/%d/%Y %H:%M:%S")

In [6]:
nomFormador = []
cedFormador = []
for index, row in DATOS.iterrows():
    dfFormador = INFO_FORMADORES[INFO_FORMADORES['CUENTA_ZOOM_UIS_DEFINITIVA_2022'] == row['E-mail del usuario']]
    nomFormador.append(dfFormador['NOMBRE_DOCENTE'].iloc[0])
    cedFormador.append(dfFormador['DOCENTE'].iloc[0])
DATOS['NOMBRE_DOCENTE'] = nomFormador
DATOS['CEDULA_DOCENTE'] = cedFormador
    

In [7]:
DATOS = DATOS[[
    'CEDULA_DOCENTE',
    'NOMBRE_DOCENTE',
    'E-mail del usuario',
    'Hora de inicio',
    'Hora de finalización',
    'Duración (minutos)',
]]

In [8]:
ced = DATOS['CEDULA_DOCENTE'].iloc[0]
testIni = DATOS['Hora de inicio'].iloc[0]
testFin = DATOS['Hora de finalización'].iloc[0]
dia, mes, anio = testIni.day, testIni.month, testIni.year
xI = testIni 
xF = testFin 
dfTest = INFO_FORMADORES[INFO_FORMADORES['DOCENTE'] == ced]
grupo = -1
for index, row in dfTest.iterrows():    
    cI = pd.to_datetime(f'{anio}-{mes}-{dia} ' + str(row['HORA_INICIAL_'])) - timedelta(minutes = 15)
    cF = pd.to_datetime(f'{anio}-{mes}-{dia} ' + str(row['HORA_FINAL_'])) + timedelta(minutes = 15)
    print (cI, xI, cF, xF)
    print ((cI <= xI) and (xF <= cF))
    if ((cI <= xI) and (xF <= cF)):
        grupo = row['GRUPO']
        break
    elif ((cI <= xI) and (xF <= testIni + timedelta(hours=1))):
        grupo = row['GRUPO']
        break
print(grupo)

2022-10-03 17:45:00 2022-10-03 17:59:25 2022-10-03 20:15:00 2022-10-03 22:03:35
False
2022-10-03 19:45:00 2022-10-03 17:59:25 2022-10-03 22:15:00 2022-10-03 22:03:35
False
-1


In [9]:
def matchGrupo(horaIni, horaFin, df, tolerancia = 15):
    dia, mes, anio = horaIni.day, horaIni.month, horaIni.year
    for index, row in df.iterrows():
        cotaInferior = pd.to_datetime(f'{anio}-{mes}-{dia} ' + str(row['HORA_INICIAL_'])) - timedelta(minutes=tolerancia)
        cotaSuperior = pd.to_datetime(f'{anio}-{mes}-{dia} ' + str(row['HORA_FINAL_'])) + timedelta(minutes=tolerancia)
        if ((cotaInferior <= horaIni) and (horaFin <= cotaSuperior)): # Normal
            return row['GRUPO']
        elif ((cotaInferior <= horaIni) and (horaFin <= cotaSuperior + timedelta(hours = 1))): # Termina muy tarde
            return row['GRUPO']
        elif ((cotaInferior - timedelta(hours = 1) <= horaIni) and (horaFin <= cotaSuperior)): # Inicia muy temprano
            return row['GRUPO']
    return -1

In [10]:
DATOS_COPIA = DATOS.copy()
adicionales = []
for index, row in DATOS_COPIA.iterrows():
    hIni = row['Hora de inicio']
    hFin = row['Hora de finalización']
    if (not(str(row['Hora de inicio'].date())) in DIAS_LABORALES) or row['Duración (minutos)'] < 235 or row['Duración (minutos)'] > 500:
        continue
    nIni = hFin - timedelta(hours = 2)
    nDur = row['Duración (minutos)'] // 2
    DATOS_COPIA['Hora de finalización'].loc[index] = nIni
    DATOS_COPIA['Duración (minutos)'].loc[index] = nDur
    adicionales.append([row['CEDULA_DOCENTE'], row['NOMBRE_DOCENTE'], row['E-mail del usuario'], nIni, row['Hora de finalización'], nDur])

tam = len(DATOS_COPIA.index)
for nuevo in adicionales:
    DATOS_COPIA.loc[tam] = nuevo
    tam += 1

        


C:\Users\radou\AppData\Local\Temp\ipykernel_21908\3925969211.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATOS_COPIA['Hora de finalización'].loc[index] = nIni
C:\Users\radou\AppData\Local\Temp\ipykernel_21908\3925969211.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATOS_COPIA['Duración (minutos)'].loc[index] = nDur


In [11]:
grupos = []
for index, row in DATOS_COPIA.iterrows():
    if not(str(row['Hora de inicio'].date())) in DIAS_LABORALES:
        grupos.append('EXTRA')
        continue
    if row['Duración (minutos)'] > 500:
        print ('Duración demasiado alta')
        print (row)
        grupo = input('Escriba el grupo al que pertenece esta sesión: ')
        grupos.append(grupo)
        continue
    ced = row['CEDULA_DOCENTE']
    dfFormador = INFO_FORMADORES[INFO_FORMADORES['DOCENTE'] == ced]
    grupo = matchGrupo(row['Hora de inicio'], row['Hora de finalización'], dfFormador, 20)
    if grupo == -1:
        print ('\n No se pudo determinar: ')
        print (row)
        grupo = input('Escriba el grupo al que pertenece esta sesión: ')
    grupos.append(grupo)


 No se pudo determinar: 
CEDULA_DOCENTE                               1049647979
NOMBRE_DOCENTE          CARLOS SANTIAGO BENITEZ MALAVER
E-mail del usuario                   zoom862@uis.edu.co
Hora de inicio                      2022-10-20 18:18:28
Hora de finalización                2022-10-20 22:07:35
Duración (minutos)                                  230
Name: 12, dtype: object
Escriba el grupo al que pertenece esta sesión: U37

 No se pudo determinar: 
CEDULA_DOCENTE                               1049647979
NOMBRE_DOCENTE          CARLOS SANTIAGO BENITEZ MALAVER
E-mail del usuario                   zoom862@uis.edu.co
Hora de inicio                      2022-11-09 18:18:13
Hora de finalización                2022-11-09 22:06:03
Duración (minutos)                                  228
Name: 26, dtype: object
Escriba el grupo al que pertenece esta sesión: U37

 No se pudo determinar: 
CEDULA_DOCENTE                               1049647979
NOMBRE_DOCENTE          CARLOS SANTIAGO BENI

Escriba el grupo al que pertenece esta sesión: EXTRA

 No se pudo determinar: 
CEDULA_DOCENTE                         1098665299
NOMBRE_DOCENTE          ALEX DAVID MORALES ACOSTA
E-mail del usuario             zoom856@uis.edu.co
Hora de inicio                2022-10-12 15:50:45
Hora de finalización          2022-10-12 16:46:49
Duración (minutos)                             57
Name: 404, dtype: object
Escriba el grupo al que pertenece esta sesión: EXTRA

 No se pudo determinar: 
CEDULA_DOCENTE                         1098665299
NOMBRE_DOCENTE          ALEX DAVID MORALES ACOSTA
E-mail del usuario             zoom856@uis.edu.co
Hora de inicio                2022-11-04 10:11:39
Hora de finalización          2022-11-04 11:22:55
Duración (minutos)                             72
Name: 420, dtype: object
Escriba el grupo al que pertenece esta sesión: EXTRA

 No se pudo determinar: 
CEDULA_DOCENTE                          1098798386
NOMBRE_DOCENTE          SILVIA JULIANA  MORENO ROA
E-mail del 

In [12]:
DATOS_COPIA['GRUPO'] = grupos

In [13]:
DATOS_COPIA.to_excel('./OUTPUT/CONSOLIDADO_DURACIONES_ZOOM_' + str(date.today()) + '.xlsx', index = False)

In [14]:
cursos = DATOS_COPIA['GRUPO'].unique()
resultados = {
    'CEDULA_DOCENTE': [],
    'NOMBRE_DOCENTE': [],
    'CURSO': [],
    'HORAS_ACUMULADAS': [],
    'FLAG': [],
    'RESTANTE': []
}
for curso in cursos:
    if curso == 'EXTRA' or curso == 'ERROR':
        continue
    dfCurso = DATOS_COPIA[DATOS_COPIA['GRUPO'] == curso]
    resultados['CEDULA_DOCENTE'].append(dfCurso['CEDULA_DOCENTE'].iloc[0])
    resultados['NOMBRE_DOCENTE'].append(dfCurso['NOMBRE_DOCENTE'].iloc[0])
    resultados['CURSO'].append(curso)
    duracionHoras = dfCurso['Duración (minutos)'].sum() / 60
    resultados['HORAS_ACUMULADAS'].append(duracionHoras)
    resultados['FLAG'].append('x' if duracionHoras < 45.5 else '')
    resultados['RESTANTE'].append(50-duracionHoras)

In [15]:
pd.DataFrame(resultados).to_excel('./OUTPUT/RESULTADOS_' + str(date.today()) + '.xlsx', index = False)